In [3]:
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('C:/Users/timei/Documents/Datalogi/Semester6/Bachelor/pm4py-dcr') # working directory should be pm4py-dcr (the one behind notebooks)
print(os.getcwd())

C:\Users\timei\Documents\Datalogi\Semester6\Bachelor\pm4py-dcr\notebooks
C:\Users\timei\Documents\Datalogi\Semester6\Bachelor\pm4py-dcr


In [4]:
import pm4py
from copy import deepcopy

In [5]:
from pm4py.objects.dcr import semantics as dcr_semantics
from pm4py.objects.dcr.importer import importer as dcr_importer
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
from pm4py.algo.discovery.dcr_discover import algorithm as dcr_alg
from pm4py.objects.dcr import enforcement

In [21]:
dcr = {
    'events': {'release', 'archive', 'delete', 'readmit', 'unarchive'},
    'labels': set(),
    'labelMapping': set(),
    'conditionsFor': {'delete': {'release'}, 'unarchive':{'release'}, 'archive':{'release'}},  # this should be a dict with events as keys and sets as values
    'milestonesFor': {'delete': {'archive'}},
    'responseTo': {},
    'noResponseTo': {},
    'includesTo': {'release':{'archive', 'delete', 'unarchive'}},
    'excludesTo': {'readmit':{'unarchive', 'archive', 'delete'}},
    'conditionsForDelays': {'unarchive':{('archive', 2920)}},  # this should be a dict with events as keys and tuples as values
    'responseToDeadlines': {'release':{('archive', 14), ('delete', 14)}},
    'marking': {'executed': set(),
                'included': {'release', 'archive', 'delete', 'readmit', 'unarchive'},
                'pending': set(),
                'executedTime': {}, # Gives the time since a event was executed
                'pendingDeadline': {} # The deadline until an event must be executed 
                }
}

In [7]:
dcr_semantics.enabled(dcr)

{'readmit', 'release'}

In [8]:
dcr_semantics.is_accepting(dcr)

True

In [22]:
dcr_semantics.execute('release', dcr) # here you execute the event
print(dcr_semantics.enabled(dcr)) # now these are the newly enabled events
print(dcr['marking']['pendingDeadline'])
print(dcr['marking']['executedTime'])

{'release', 'archive', 'readmit'}
{'archive': 14, 'delete': 14}
{'release': 0}


In [23]:
dcr_semantics.execute('archive', dcr) # here you execute the event
print(dcr_semantics.enabled(dcr))
dcr_semantics.execute('delete', dcr)
print(dcr['marking']['pendingDeadline'])
print(dcr['marking']['executedTime'])

{'release', 'archive', 'delete', 'readmit'}
{}
{'release': 0, 'archive': 0, 'delete': 0}


In [25]:
dict_exe = dcr_semantics.create_max_executed_time_dict(dcr)
dcr_semantics.time_step(2919, dcr, dict_exe)
print(dcr['marking']['pendingDeadline'])
print(dcr['marking']['executedTime'])
print(dcr_semantics.enabled(dcr))

{}
{'release': 0, 'archive': 2920, 'delete': 0}
{'release', 'unarchive', 'archive', 'delete', 'readmit'}


In [14]:
e = enforcement.Enforcement_mechanisme(dcr)
e.peform_controllable_event("release")
print(dcr_semantics.enabled(e.dcr))

{'readmit', 'release', 'archive'}


In [47]:
print(e.iGraph.is_cyclic())
print(e.iGraph.check_for_responses())
print(e.iGraph.check_for_delays())
print(e.iGraph.executeGraph)
print(e.iGraph.oppGraph)

False
True
True
{'unarchive': ['release', 'archive'], 'archive': ['release'], 'delete': ['release', 'archive']}
{'delete': ['release', 'archive'], 'unarchive': ['release', 'archive'], 'archive': ['release']}


In [26]:
e.check_urgent_deadlines()

In [12]:
print(dcr_semantics.find_next_deadline(dcr))

14


In [ ]:
g = enforcement.IGraph(dcr)
print(g.reachableGraph)
print(g.graph)
print(g.topologicalSort())
print(g.executeGraph)